In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config
import data_util.preprocess

importlib.reload(data_util.preprocess)
importlib.reload(train)
importlib.reload(model)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from train import *
from evaluate import *
from model import *
from train_util import *
from data_util.data import *
from data_util.batcher import *
from data_util.preprocess import *

In [10]:
task_train = pd.read_csv(os.path.join(config.sum_path, 'task_train.tsv'), sep = '\t')
wiki_train = pd.read_csv(os.path.join(config.sum_path, 'wiki_train.tsv'), sep = '\t')
wiki_val   = pd.read_csv(os.path.join(config.sum_path, 'wiki_val.tsv'  ), sep = '\t')
task_val   = pd.read_csv(os.path.join(config.sum_path, 'task_val.tsv'  ), sep = '\t')

In [18]:
task_val   = pd.read_csv(os.path.join(config.sum_path, 'task_val.tsv'  ), sep = '\t')

In [11]:
# check sizes
print(task_train.shape)
print(wiki_train.shape)
print(wiki_val.shape)

(1956, 3)
(917986, 3)
(300, 3)


In [12]:
# create vocabulary
vocab = Vocab(os.path.join(config.vocab_path, 'vocab3.txt'));

Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated w

In [14]:
# argument namespace
opt = Namespace(
    train_mle = "yes", 
    train_rl = "no", 
    mle_weight = 1., 
    load_model = None,
    new_lr = None, 
    rl_weight = 0.)

# create train and val batchers
wiki_batcher = TaskBatcher(
    examples=wiki_train[:1000].to_dict('records'),
    vocab=vocab,
    mode='train',
    batch_size=16,
    single_pass=False
)
wiki_val_batcher = TaskBatcher( # Batching obj
    examples=wiki_val.to_dict('records')[:200],
    vocab=vocab, 
    mode='train', 
    batch_size=50, 
    single_pass=False
)

In [16]:
# train_processor = TaskTrain(vocab, task_batcher, opt, TaskModel, val_task_batcher)
wiki_trainer = TaskTrain(vocab, wiki_batcher, opt, TaskModel, wiki_val_batcher)
# load pre-trained embedding weights
wiki_trainer.model.load_embeddings("embedding_43861_200.tar")

In [17]:
# train
mle_losses = wiki_trainer.trainIters(n_iters=15, report_every=1, save_every = 5)

iter: 1 mle_loss: 8.751 mle_loss_val: -100.0000
iter: 2 mle_loss: 8.145 mle_loss_val: -100.0000
iter: 3 mle_loss: 8.132 mle_loss_val: -100.0000
iter: 4 mle_loss: 7.853 mle_loss_val: -100.0000
iter: 5 mle_loss: 7.729 mle_loss_val: 7.8103
model saved at: 
 data/saved_models/0000005.tar
iter: 6 mle_loss: 8.068 mle_loss_val: 7.8103
iter: 7 mle_loss: 7.243 mle_loss_val: 7.8103
iter: 8 mle_loss: 7.274 mle_loss_val: 7.8103
iter: 9 mle_loss: 7.067 mle_loss_val: 7.8103
iter: 10 mle_loss: 6.161 mle_loss_val: 5.9920
model saved at: 
 data/saved_models/0000010.tar
iter: 11 mle_loss: 5.582 mle_loss_val: 5.9920
iter: 12 mle_loss: 6.153 mle_loss_val: 5.9920
iter: 13 mle_loss: 6.236 mle_loss_val: 5.9920
iter: 14 mle_loss: 6.432 mle_loss_val: 5.9920
iter: 15 mle_loss: 5.813 mle_loss_val: 5.8314
model saved at: 
 data/saved_models/0000015.tar
iter: 16 mle_loss: 6.197 mle_loss_val: 5.8314


In [19]:
mle_losses

{'train': [(1, 8.751328468322754),
  (2, 8.145495414733887),
  (3, 8.132192611694336),
  (4, 7.853020668029785),
  (5, 7.729152679443359),
  (6, 8.067704200744629),
  (7, 7.242686748504639),
  (8, 7.274189472198486),
  (9, 7.066506385803223),
  (10, 6.160511016845703),
  (11, 5.582155227661133),
  (12, 6.152600288391113),
  (13, 6.235994338989258),
  (14, 6.4322710037231445),
  (15, 5.812689781188965),
  (16, 6.197237968444824)],
 'val': [(5, 7.81031060218811),
  (10, 5.992038726806641),
  (15, 5.8313528299331665)]}

## Fine Tune the Model to Task data

In [25]:
# Retrain namespace
opt_retrain = Namespace(
    train_mle = "yes", 
    train_rl = "no", 
    mle_weight = 1., 
    load_model = '0000015.tar',
    new_lr = None, 
    rl_weight = 0.)

# create train and val batchers for task data
task_batcher = TaskBatcher(
    examples=task_train.to_dict('records'),
    vocab=vocab,
    mode='train',
    batch_size=16,
    single_pass=False
)
task_val_batcher = TaskBatcher( # Batching obj
    examples=task_val.to_dict('records')[:200],
    vocab=vocab, 
    mode='train', 
    batch_size=50, 
    single_pass=False
)

# create (re)trainer
task_trainer = TaskTrain(vocab, task_batcher, opt_retrain, TaskModel)

Loaded model at data/saved_models/0000015.tar


In [26]:
config.save_model_path = 'data/saved_models2/'
mle_losses_task = task_trainer.trainIters(n_iters=3, report_every=1, save_every = 3)

iter: 1 mle_loss: 5.016 mle_loss_val: -100.0000
iter: 2 mle_loss: 4.740 mle_loss_val: -100.0000
iter: 3 mle_loss: 4.947 mle_loss_val: -100.0000
model saved at: 
 data/saved_models2//0000003.tar
iter: 4 mle_loss: 4.371 mle_loss_val: -100.0000


In [28]:
mle_losses_task

{'train': [(1, 5.0163164138793945),
  (2, 4.740086078643799),
  (3, 4.947429656982422),
  (4, 4.370893478393555)],
 'val': []}

## Generating Summaries 

In [46]:
load_model = os.path.join(config.log_root, "data/saved_models/0000015.tar") # model directory

# new batcher for evaluation
wiki_eval_batcher = TaskBatcher(
    examples=wiki_val.to_dict('records'),
    vocab=vocab, 
    mode='train', 
    batch_size=50,
    single_pass=True)

task_eval_batcher = TaskBatcher(
    examples=task_val.to_dict('records'),
    vocab=vocab, 
    mode='train', 
    batch_size=50,
    single_pass=True)

evaluator = TaskEvaluate(vocab, task_eval_batcher, TaskModel, load_model) # Evaluation object

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# check model parameter count
count_parameters(evaluator.model)

41450470

In [50]:
decoded_sents, ref_sents, task_sents, context_sents = evaluator.evaluate_batch()

Summarizing Batch...
Summarizing Batch...
Summarizing Batch...
Summarizing Batch...
Summarizing Batch...
example_generator completed reading all examples. No more data.


Exception in thread Thread-30:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 445, in text_generator
    example = next(example_generator)
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 426, in fill_example_queue
    context, task, summary = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



example_generator completed reading all examples. No more data.


Exception in thread Thread-28:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 445, in text_generator
    example = next(example_generator)
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 426, in fill_example_queue
    context, task, summary = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



Summarizing Batch...


In [51]:
len(ref_sents)

300

In [52]:
len(decoded_sents)

300

In [44]:
pd.set_option('display.max_colwidth', -1)

In [53]:
df = pd.DataFrame({'task': task_sents,'ref': ref_sents, 'decoded': decoded_sents})
df.sample(10)

,task,ref,decoded
242,Please use the Alberta Phone Log next to the PX trade app.,use the Alberta phone log,the the the
23,"But, I'm still here; give me a call when you get a chance - 1586.",call SENDER,
87,"Christian, could you coordinate the writing of this letter.",coordinate writing of letter,the the
163,"Please call 1-800-2924-380 , or look on the mailing label from the envelope your GTR was mailed in.",call 1-800-2924 - 380,
275,Could you give me a call at your convenience?,call SENDER,
287,"If okay, please complete and send.",complete document and return to SENDER,please
247,Please take a moment to respond to the attached survey.,respond to survey,to to to to to
90,"Aha, could you provide a column including the approver.",provide column,provide the the
246,Can you check Mike Swerzbin for the following deals through Prebon:,check Mike Swerzbin for these deals,check the the
116,Could you forward them to me by email?,forward these documents to SENDER,
